<a href="https://colab.research.google.com/github/lloyd31/CNNDetection/blob/master/CNNBasedDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Clone the CNN repo
!git clone https://github.com/lloyd31/CNNDetection.git

Cloning into 'CNNDetection'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 637 (delta 1), reused 7 (delta 1), pack-reused 626
Receiving objects: 100% (637/637), 6.37 MiB | 20.39 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
ls -l

total 8
drwxr-xr-x 9 root root 4096 Mar 14 15:05 CNNDetection/
drwxr-xr-x 1 root root 4096 Mar  9 14:48 sample_data/


In [ ]:
cd ./CNNDetection

/content/CNNDetection


In [ ]:
pwd

'/content/CNNDetection'

In [ ]:
#Run the installation requirements if running on a local development set up
#pip install -r requirements.txt

In [ ]:
!python --version

Unknown option: -e
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
